
# Onderzoeksvraag 1: In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?


In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
# nltk.download()

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

In [ ]:
wine = pd.read_csv('redwine.csv', delimiter=';')
chemColNames = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
chem = wine[chemColNames]

colErrorPairs = {
    'density'    : [' . '],
    'citric acid': [' - ',' -   '],
    'alcohol'    : ['100.333.333.333.333','11.066.666.666.666.600','956.666.666.666.667','923.333.333.333.333']}

for colName in colErrorPairs:
    for faultyString in colErrorPairs[colName]:
        wine[colName] = wine[colName].replace(faultyString,np.nan)
        
wine['alcohol'] = wine['alcohol'].astype(float)
wine['density'] = wine['density'].astype(float)
wine['citric acid'] = wine['citric acid'].astype(float)

wineZscore = wine.copy()

for col in chemColNames:
        wineZscore[col] = (wine[col] - wine[col].mean())/wine[col].std(ddof=0)

## Lineaire regressie

Voor de eerste verplichte onderzoeksvraag gaan we supervised machine learning gebruiken. We willen de score van een wijn gaan voorspellen aan de hand van de chemische samenstelling. Dit gaan we doen met behulp van lineaire regressie. 

__1. Kies een modeltype__ 

In [ ]:
from sklearn.linear_model import LinearRegression

__2. Kies de hyperparameters__


__3. Organiseer de data (feature matrix, target vector)__

Omdat we op basis van de chemische samenstelling willen gaan voorspellen, worden dit onze feature variabelen. 

In [ ]:
chem.head()

### Standaardiseren

Omdat de waardes van de features nogal uit elkaar liggen gaan we ze eerst standaardiseren met behulp van de Z Score. Zo zorgen we ervoor dat elke waarde tussen de -1 en 1 komt te liggen, zodat alle kolommen even zwaar mee tellen. 

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates']

In [ ]:
X = wineZscore[features]
X.head()

De target variabele is bij deze onderzoeksvraag de points, dit is namelijk de variabele die we willen voorspellen. 

In [ ]:
y = wine.points
y.head()

__4. Creeër een training- en validatie set.__

Nu gaan we een training en test set creëeren. Door het opvullen van lege waarden met de gemiddelden verhogen we linreg.score() een beetje, deze komt later aan bod. 

In [ ]:
X.fillna(X.mean(),inplace=True)
y.fillna(y.mean(),inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

De wijn dataset bestaat uit 2465 records, we gebruiken ongeveer 75% van de waardes voor de trainingset en de overige 25% voor de testset. 

In [ ]:
len(wine)

In [ ]:
len(X_train)
(X_train.shape, y_train.shape)

In [ ]:
len(X_test)

In [ ]:
linreg = LinearRegression()

Het model gaat nu leren aan de hand van de data uit de trainingsset. 

In [ ]:
linreg.fit(X_train, y_train)

We gaan nu de kwaliteit van het model berekenen op basis van de data uit de testset. 

In [ ]:
linreg.score(X_test,y_test)

De score van het model is 0.04, dat deze score zo dicht bij 0 ligt wil eigenlijk zeggen dat er geen lineair verband te leggen is tussen de features en de target variabelen. 

In [ ]:
y_pred = linreg.predict(X_test)

De mean squared error is de gemiddelde afwijking van de voorspelde waarden. Je wilt dit getal zo laag mogelijk hebben omdat het dus een error aangeeft. 

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

### Correlatie

Correlatie toont de samenhang tussen variabelen. Hiervoor gebruiken we wederom de Zscore. Bij een grote correlatie kun je aan de hand van de ene variabele heel goed de andere voorspellen, bij een kleine correlatie gaat dit (bijna) niet. 

Correlatie kan positief en negatief zijn. Positieve correlatie wil zeggen dat wanneer X toeneemt, Y ook toeneemt. Negatieve correlatie wil zeggen dat wanneer X toeneemt, Y afneemt. Omdat een correlatie tussen de -0.4 en 0.4 zo klein is, kun je eigenlijk niet echt spreken van samenhang. 

In [ ]:
wineZscore.corr()

__Correlatie tussen de variabelen__

We gaan kijken naar de samenhang tussen variabelen onderling. Zowel featurue variabelen met elkaar, als target en feature variabelen. Hieronder zien we welke combinatie van variabelen met meest correleren met elkaar.  

In [ ]:
c = wineZscore.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")
so[172:182]

Points en price zijn redelijk gecorreleert, dit kan interessant zijn maar helaas is price niet een feature variabelen. Voor deze onderzoeksvraag hebben we daar dus helaas niet heel veel aan. Wel zien we hier enigzins een positieve correlatie. 

In [ ]:
plt.scatter(wine.points, wine.price)

Verder zien we in de tabel hierboven nog dat fixed acidity en pH sterk gecorreleert zijn. Bij sterke correlatie tussen twee feature variabelen kun je gaan nadenken of dit wijst op redundantie, misschien heb je maar 1 van de twee variabelen nodig als ze toch bijna hetzelfde aangeven. Ook kunnen de twee variabelen juist elkaar tegen werken. 

In [ ]:
fa = wine['fixed acidity']

In [ ]:
plt.scatter(wine.pH, fa)

# Conclusie

Tussen de feature en de target variabelen is weinig samenhang te vinden. Is het mogelijk om de score van de wijn te voorspellen aan de hand van de chemische samenstelling? Nee, eigenlijk niet. Misschien kunnen we beter de score van de wijn gaan voorspellen aan de hand van de prijs. 